<a href="https://colab.research.google.com/github/youssefjedidi/Aircraft_Noise_Predictor/blob/main/aircraft_noise_predictor_ML_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Regression models

## Importing the libraries

In [53]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [54]:
dataset = pd.read_csv('aircraft_noise.csv')
X = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values

In [55]:
print(X)

[['LAmax' 'A' 1500 'Piston']
 ['LAmax' 'A' 1600 'Piston']
 ['LAmax' 'A' 27 'Piston']
 ...
 ['SEL' 'D' 28000 'Jet']
 ['EPNL' 'D' 32000 'Jet']
 ['SEL' 'D' 32000 'Jet']]


## Encoding categorical data

In [56]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [  1 , 0 ,3 ])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [57]:
print(X)

[[1.0 0.0 0.0 ... 1.0 0.0 1500]
 [1.0 0.0 0.0 ... 1.0 0.0 1600]
 [1.0 0.0 0.0 ... 1.0 0.0 27]
 ...
 [0.0 1.0 0.0 ... 0.0 0.0 28000]
 [0.0 1.0 1.0 ... 0.0 0.0 32000]
 [0.0 1.0 0.0 ... 0.0 0.0 32000]]


## Splitting the dataset into the Training set and Test set

In [58]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Multiple Linear Regression


### Training the Multiple Linear Regression model on the Training set

In [59]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

### Predicting the Test set results

In [60]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[63.48 59.5 ]
 [56.31 55.8 ]
 [71.04 86.5 ]
 ...
 [72.07 73.5 ]
 [71.65 65.6 ]
 [69.45 61.5 ]]


### Testing accuracy

In [61]:
def mean_absolute_percentage_error(y_true, y_pred):
    return (np.mean(np.abs((y_true - y_pred) / y_true)) * 100)

mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(regressor.predict([[1, 0, 0, 1 ,0 ,0 , 0,0 ,1 , 300]]))

from sklearn.metrics import r2_score

# Assuming y_test contains the actual target values and y_pred contains the predicted values
# Calculate R squared
r2 = r2_score(y_test, y_pred)

# Print MSE
print(f" R squared  (r2): {r2:.2f}")

Mean Absolute Percentage Error (MAPE): 9.94%
[43.62]
 R squared  (r2): 0.55


## Polynomial Regression

### Training

In [62]:
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree = 3)
X_poly = poly_reg.fit_transform(X_train)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y_train)

LinearRegression()

### Predecting the Test set Results

In [63]:
y_poly = lin_reg_2.predict(poly_reg.fit_transform(X_test))
np.set_printoptions(precision=2)
print(np.concatenate((y_poly.reshape(len(y_poly),1), y_test.reshape(len(y_test),1)),1))

[[65.6  59.5 ]
 [57.72 55.8 ]
 [72.81 86.5 ]
 ...
 [74.62 73.5 ]
 [73.72 65.6 ]
 [69.62 61.5 ]]


### Testing accuracy

In [64]:

mape = mean_absolute_percentage_error(y_test, y_poly)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(lin_reg_2.predict(poly_reg.fit_transform([[1, 0, 0, 1 ,0 ,0 , 0,0 ,1 , 300]])))

from sklearn.metrics import r2_score

# Assuming y_test contains the actual target values and y_pred contains the predicted values
# Calculate R squared
r2_poly = r2_score(y_test, y_poly)

# Print MSE
print(f" R squared  (r2): {r2_poly:.2f}")

Mean Absolute Percentage Error (MAPE): 9.06%
[45.92]
 R squared  (r2): 0.63


## Support Vector Regression (SVR)

### Feature Scaling

In [65]:
from sklearn.preprocessing import StandardScaler

# Reshape input data to 2D array
X_reshaped = X[:, 3].reshape(-1, 1)  # Reshape only the fourth column
X_test_reshaped = X_test[:, 3].reshape(-1, 1)

# Initialize StandardScaler for the fourth column
sc_X = StandardScaler()

# Fit and transform the fourth column
X_scaled = X.copy()  # Create a copy of X to keep other columns unchanged
X_scaled[:, 3] = sc_X.fit_transform(X_reshaped).flatten()  # Scale and update the fourth column

X_test_scaled = X_test.copy()
X_test_scaled[:, 3] = sc_X.fit_transform(X_test_reshaped).flatten()

# Initialize StandardScaler for the target variable
sc_y = StandardScaler()
y_scaled = sc_y.fit_transform(y.reshape(-1, 1))  # Fit and transform the target variable
y_test_scaled = sc_y.fit_transform(y_test.reshape(-1, 1))

# Now X_scaled and y_scaled are scaled versions of X and y using StandardScaler
print("Scaled X:")
print(X_scaled)
print("\nScaled y:")
print(y_scaled)

Scaled X:
[[1.0 0.0 0.0 ... 1.0 0.0 1500]
 [1.0 0.0 0.0 ... 1.0 0.0 1600]
 [1.0 0.0 0.0 ... 1.0 0.0 27]
 ...
 [0.0 1.0 0.0 ... 0.0 0.0 28000]
 [0.0 1.0 1.0 ... 0.0 0.0 32000]
 [0.0 1.0 0.0 ... 0.0 0.0 32000]]

Scaled y:
[[-3.06]
 [-2.97]
 [-2.9 ]
 ...
 [ 3.36]
 [ 3.47]
 [ 3.5 ]]


### Training the SVR model

In [66]:
from sklearn.svm import SVR
sv_regressor = SVR(kernel = 'rbf')
sv_regressor.fit(X_scaled, y_scaled)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR()

### Predicting

In [67]:
# Perform predictions using the SVR model on the scaled test features
y_svr_scaled = sv_regressor.predict(X_test_scaled)

# Inverse transform the scaled predictions to get them back to the original scale
y_svr = sc_y.inverse_transform(y_svr_scaled.reshape(-1, 1))

np.set_printoptions(precision=2)
print(np.concatenate((y_svr.reshape(len(y_svr),1), y_test.reshape(len(y_test),1)),1))

[[63.34 59.5 ]
 [63.7  55.8 ]
 [67.9  86.5 ]
 ...
 [69.   73.5 ]
 [68.7  65.6 ]
 [70.72 61.5 ]]


### Testing accuracy

In [68]:

mape = mean_absolute_percentage_error(y_test, y_svr)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

from sklearn.metrics import r2_score

# Assuming y_test contains the actual target values and y_pred contains the predicted values
# Calculate R squared
r2_poly = r2_score(y_test, y_svr)

# Print MSE
print(f" R squared  (r2): {r2_poly:.2f}")

Mean Absolute Percentage Error (MAPE): 18.25%
 R squared  (r2): 0.29


## Decision Tree

### Training the Decision Tree model on the Training set

In [69]:
from sklearn.tree import DecisionTreeRegressor
tree_regressor = DecisionTreeRegressor(random_state = 0)
tree_regressor.fit(X_test, y_test)

DecisionTreeRegressor(random_state=0)

###Predicting

In [70]:
y_tree = tree_regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_tree.reshape(len(y_tree),1), y_test.reshape(len(y_test),1)),1))

[[59.5 59.5]
 [55.8 55.8]
 [83.4 86.5]
 ...
 [73.5 73.5]
 [82.  65.6]
 [61.5 61.5]]


### Testing accuracy

In [71]:
mape = mean_absolute_percentage_error(y_test, y_tree)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(tree_regressor.predict([[1, 0, 0, 1 ,0 ,0 , 0,0 ,1 , 300]]))

from sklearn.metrics import r2_score

# Assuming y_test contains the actual target values and y_pred contains the predicted values
# Calculate R squared
r2 = r2_score(y_test, y_tree)

# Print MSE
print(f" R squared  (r2): {r2:.2f}")

Mean Absolute Percentage Error (MAPE): 2.27%
[40.4]
 R squared  (r2): 0.93


## Random Forest

### Training the Random Forest model on the Training set

In [72]:
from sklearn.ensemble import RandomForestRegressor
forest_regressor = RandomForestRegressor(n_estimators = 222, random_state = 0)
forest_regressor.fit(X_test, y_test)

RandomForestRegressor(n_estimators=222, random_state=0)

### Predicting

In [73]:
y_forest = forest_regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_forest.reshape(len(y_forest),1), y_test.reshape(len(y_test),1)),1))

[[61.19 59.5 ]
 [56.09 55.8 ]
 [83.04 86.5 ]
 ...
 [72.69 73.5 ]
 [81.56 65.6 ]
 [62.19 61.5 ]]


### Testing accuracy

In [74]:
mape = mean_absolute_percentage_error(y_test, y_forest)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(forest_regressor.predict([[1, 0, 0, 1 ,0 ,0 , 0,0 ,1 , 300]]))

from sklearn.metrics import r2_score

# Assuming y_test contains the actual target values and y_pred contains the predicted values
# Calculate R squared
r2 = r2_score(y_test, y_forest)

# Print MSE
print(f" R squared  (r2): {r2:.2f}")

Mean Absolute Percentage Error (MAPE): 3.93%
[42.75]
 R squared  (r2): 0.91
